### "데이터 문화가 되다"  
2021(제2회) NH투자증권 빅데이터 경진대회 BASELINE

BASELINE을 통해 간단한 데이터 소개와 함께 힌트를 제공하려고 합니다.  
이번 대회는 stk_hld_train.csv 파일의 hold_d를 Y값으로 모델을 학습시키고  
stk_hld_test.csv 파일의 hold_d를 예측하는 과제를 풀어야 합니다.

In [3]:
train.head() # stk_hld_train.csv

,act_id,iem_cd,byn_dt,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324


In [4]:
test.head() # stk_hld_test.csv

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0


이미 눈치채신 분들도 있겠지만 stk_hld_test 데이터에는 stk_hld_train 데이터에 없는 column이 있습니다.  
바로 "hist_d" column입니다. "hist_d"는 stk_hld_test 데이터에 속한 고객이 2021년 이전에 주식을 보유한 기간입니다.

#### *Hint  
<span style="color:red"><U>**Hold_d(보유기간) – hist_d(‘20년 12월 31일까지의 최근 보유기간) ≤ 146**</U></span>  
  
2021년 데이터 기간은 ‘21년 1월 1일 ~ ‘21년 7월 31일로 제한했습니다.  
  
따라서 학습 데이터 셋에서 다양한 기준으로 hist_d column을 만들어 새로운 feature로 사용해 보시는 것을 추천드립니다.

#### *Hint

추가적으로 새로운 feature로 사용할 수 있는 정보에는 고객의 이전 주식 보유이력이 있습니다.(past_d로 칭하겠습니다.)  
  
stk_bnc_hist.csv 데이터를 살펴보면 stk_hld_test.csv 고객들의 이전 주식 보유이력(past_d)을 확인할 수 있습니다.  
  
stk_hld_train.csv도 마찬가지로 past_d column을 만들 수 있을 것으로 보입니다.

### 라이브러리 불러오기

In [1]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

### 데이터 불러오기

In [2]:
cus = pd.read_csv("open/cus_info.csv")
iem = pd.read_csv("open/iem_info.csv")
hist = pd.read_csv("open/stk_bnc_hist.csv")
train = pd.read_csv("open/stk_hld_train.csv")
test = pd.read_csv("open/stk_hld_test.csv")

submission = pd.read_csv("open/sample_submission.csv")

### 데이터 전처리

In [3]:
# 앞서 언급한 것처럼 베이스라인에서는 "hist_d" column을 임의로 생성하여 모델을 학습시키도록 하겠습니다.
# 베이스라인에서는 "hold_d"값, 즉 주식 보유기간의 0.6배에 해당하는 기간을 임의로 설정하여 "hist_d"를 생성하였습니다.
# 결국 모델은 "hist_d"만큼 주식을 보유 했을때의 "hold_d"를 예측하게 될 것입니다.

train["hist_d"] = train["hold_d"]*0.6
train.hist_d = np.trunc(train["hist_d"])

In [4]:
train.head(3)

,act_id,iem_cd,byn_dt,hold_d,hist_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0


In [5]:
# train과 test에 고객정보(cus_info)와 주식정보(iem_info)를 추가하겠습니다.

train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

In [6]:
train_data.head(3)

,act_id,iem_cd,byn_dt,hold_d,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0,1,9,3,2,2,9,5,5,8,GS건설,14,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0,1,9,3,2,2,9,5,5,8,삼성전자,14,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,14,2,99


In [7]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

In [8]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [9]:
train_data.head(3)

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,6.0,1,9,3,2,2,9,5,5,8,101,14,1,1,274.0,11782000.0,5000.0,43000.0
1,48.0,1,9,3,2,2,9,5,5,8,1361,14,1,1,2.0,4990000.0,5000.0,2495000.0
2,3.0,1,9,3,2,2,9,5,5,8,2530,14,2,99,786.0,14619600.0,1000.0,18600.0


In [10]:
test_data.head(3)

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,153,1,9,3,2,2,9,5,5,8,417,14,1,1,300.0,3945000.0,5000.0,13150.0
1,335,1,9,3,2,2,9,5,5,8,2231,14,3,99,198.0,2524500.0,500.0,12750.0
2,139,1,9,3,2,2,9,5,5,8,1514,14,2,99,138.0,4291800.0,500.0,31100.0


In [11]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

### 모델 학습: LGBMRegressor 사용

In [13]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 4.22398	valid_0's l2: 17.842
[1000]	valid_0's rmse: 4.07674	valid_0's l2: 16.6198
[1500]	valid_0's rmse: 4.03603	valid_0's l2: 16.2896
[2000]	valid_0's rmse: 4.01849	valid_0's l2: 16.1483
Did not meet early stopping. Best iteration is:
[1997]	valid_0's rmse: 4.01845	valid_0's l2: 16.148
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.29758	valid_0's l2: 1.6837
[1000]	valid_0's rmse: 1.23595	valid_0's l2: 1.52757
Early stopping, best iteration is:
[747]	valid_0's rmse: 1.19853	valid_0's l2: 1.43648
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.67906	valid_0's l2: 0.461122
[1000]	valid_0's rmse: 0.474997	valid_0's l2: 0.225622
[1500]	valid_0's rmse: 0.473955	valid_0's l2: 0.224634
[2000]	valid_0's rmse: 0.474531	valid_0's l2: 0.225179
Did not meet early stopping. Best iteration is:
[1714]	valid_0's rmse: 0.473498	valid_0's l2: 0

### 결과 추론

In [14]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [15]:
predict

array([254.83943129, 557.69846901, 232.3049104 , ..., 886.29888571,
        18.998117  ,   7.46568821])

In [16]:
submission["hold_d"] = np.round(predict)

### 결과물 저장

In [17]:
submission.to_csv("dacon_baseline.csv", index = False)

끝으로 2021(제2회) NH투자증권 빅데이터 경진대회는 외부데이터의 사용을 적극적으로 권장합니다.  
참가자 여러분들의 다양한 데이터와 창의적인 방법을 통해 주식 보유기간 예측 모델을 고도화시켜 주시길 바라겠습니다.